Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [3]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [4]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [5]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [6]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [7]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [8]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.547965
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.642329
Minibatch accuracy: 56.2%
Validation accuracy: 49.8%
Minibatch loss at step 100: 1.562582
Minibatch accuracy: 50.0%
Validation accuracy: 69.8%
Minibatch loss at step 150: 0.694608
Minibatch accuracy: 81.2%
Validation accuracy: 76.3%
Minibatch loss at step 200: 0.663996
Minibatch accuracy: 75.0%
Validation accuracy: 76.0%
Minibatch loss at step 250: 1.190318
Minibatch accuracy: 62.5%
Validation accuracy: 78.4%
Minibatch loss at step 300: 0.293090
Minibatch accuracy: 93.8%
Validation accuracy: 78.3%
Minibatch loss at step 350: 0.239476
Minibatch accuracy: 100.0%
Validation accuracy: 77.8%
Minibatch loss at step 400: 0.640537
Minibatch accuracy: 75.0%
Validation accuracy: 79.5%
Minibatch loss at step 450: 0.431527
Minibatch accuracy: 87.5%
Validation accuracy: 80.5%
Minibatch loss at step 500: 0.881256
Minibatch accuracy: 75.0%
Validation accuracy: 80.8%


---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [35]:
from math import ceil

batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

def calc_HW(in_size, conv_stride, pool_stride):
    layer1_conv_HW = ceil(in_size / conv_stride)
    layer1_pool_HW = ceil(layer1_conv_HW / pool_stride)
    layer2_conv_HW = ceil(layer1_pool_HW / conv_stride)
    layer2_pool_HW = ceil(layer2_conv_HW / pool_stride)
    return int(layer2_pool_HW)

def init_weight_stddev(fan_in_size):
    return np.sqrt(2.0/float(fan_in_size))

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=init_weight_stddev(patch_size*patch_size*num_channels)))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=init_weight_stddev(patch_size*patch_size*depth)))
  layer2_biases = tf.Variable(tf.zeros([depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [calc_HW(image_size, 2, 2) * calc_HW(image_size, 2, 2) * depth, num_hidden], stddev=init_weight_stddev(calc_HW(image_size, 2, 2) * calc_HW(image_size, 2, 2) * depth)))
  layer3_biases = tf.Variable(tf.zeros([num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=init_weight_stddev(num_hidden)))
  layer4_biases = tf.Variable(tf.zeros([num_labels]))

  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    relu = tf.nn.relu(conv + layer1_biases)
    hidden = tf.nn.max_pool(relu, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    relu = tf.nn.relu(conv + layer2_biases)
    hidden = tf.nn.max_pool(relu, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [36]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.418697
Minibatch accuracy: 6.2%
Validation accuracy: 9.7%
Minibatch loss at step 50: 1.437775
Minibatch accuracy: 62.5%
Validation accuracy: 62.4%
Minibatch loss at step 100: 0.958705
Minibatch accuracy: 75.0%
Validation accuracy: 74.3%
Minibatch loss at step 150: 0.603622
Minibatch accuracy: 81.2%
Validation accuracy: 76.3%
Minibatch loss at step 200: 0.816757
Minibatch accuracy: 81.2%
Validation accuracy: 77.4%
Minibatch loss at step 250: 0.901415
Minibatch accuracy: 75.0%
Validation accuracy: 74.3%
Minibatch loss at step 300: 0.226415
Minibatch accuracy: 93.8%
Validation accuracy: 77.8%
Minibatch loss at step 350: 0.202811
Minibatch accuracy: 93.8%
Validation accuracy: 77.5%
Minibatch loss at step 400: 0.593826
Minibatch accuracy: 81.2%
Validation accuracy: 78.9%
Minibatch loss at step 450: 0.348410
Minibatch accuracy: 93.8%
Validation accuracy: 80.8%
Minibatch loss at step 500: 0.767962
Minibatch accuracy: 68.8%
Validation accuracy: 80.7%
Min

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [58]:
from math import ceil, pow

batch_size = 16
num_hidden = 128
# patch_size = {'conv1':5, 'pool1':2, 'conv2':5, 'pool2':2, 'conv3':4, 'pool3':2}
# depth = {'conv1':16, 'conv2':36, 'conv3':120}
# stride = {'conv1':1, 'pool1':2, 'conv2':1, 'pool2':2, 'conv3':1, 'pool3':2}
# padding = {'conv1':'VALID', 'pool1':'SAME', 'conv2':'VALID', 'pool2':'SAME', 'conv3':'VALID', 'pool3':'SAME'}
patch_size = {'conv1':5, 'pool1':2, 'conv2':5, 'pool2':2}
depth = {'conv1':16, 'conv2':36}
stride = {'conv1':1, 'pool1':2, 'conv2':1, 'pool2':2}
padding = {'conv1':'SAME', 'pool1':'SAME', 'conv2':'SAME', 'pool2':'SAME'}
dropout_keep_prob = 0.5
starter_learning_rate = 0.1

def calc_output_size(in_size, patch, stride, padding):
    if padding == 'VALID':
        return int(ceil((in_size - patch + 1) / stride))
    else:
        return int(ceil(in_size / stride))
    
def init_weight_stddev(fan_in_size):
    return np.sqrt(2.0/float(fan_in_size))

graph = tf.Graph()

with graph.as_default():

  # Input data.
  dataset = tf.placeholder(
    tf.float32, shape=[None, image_size, image_size, num_channels], name="x")
  labels = tf.placeholder(tf.float32, shape=[None, num_labels], name="labels")
  
  
  def init_conv_weight(patch_size, in_depth, out_depth):
      return tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, in_depth, out_depth], stddev=init_weight_stddev(patch_size*patch_size*in_depth)), name="W")
    
  def init_full_weight(in_size, out_size):
    return tf.Variable(tf.truncated_normal(
      [in_size, out_size], stddev=init_weight_stddev(in_size)), name="W")
  


  def calc_conv_layer(in_data, conv_stride, pool_stride, conv_padding, pool_padding, conv_patch_size, pool_patch_size, depth, name="conv"):
    with tf.name_scope(name):
        shape = in_data.get_shape().as_list()
        weights = init_conv_weight(conv_patch_size, shape[3], depth)
        biases = tf.Variable(tf.zeros([depth]), name="B")
        conv = tf.nn.conv2d(in_data, weights, [1, conv_stride, conv_stride, 1], padding=conv_padding)
        conv_HW = calc_output_size(shape[1], conv_patch_size, conv_stride, conv_padding)
        if conv_HW >= pool_patch_size:
            conv = tf.nn.max_pool(conv, [1, pool_patch_size, pool_patch_size, 1], [1, pool_stride, pool_stride, 1], padding=pool_padding)
        return tf.nn.relu(conv + biases)

  def calc_full_layer(in_data, out_size, keep_prob, use_relu, name="fc"):
    with tf.name_scope(name):
        shape = in_data.get_shape().as_list()
        weights = init_full_weight(shape[1], out_size)
        biases = tf.Variable(tf.zeros([out_size]), name="B")
        result = tf.matmul(in_data, weights) + biases
        if use_relu:
            result = tf.nn.relu(result)
        return tf.nn.dropout(result, keep_prob)
    
   
    # Model.
  def model(data, keep_prob = 1):
    conv_layer1 = calc_conv_layer(data,
        stride['conv1'],stride['pool1'],padding['conv1'],padding['pool1'],patch_size['conv1'],patch_size['pool1'],depth['conv1'],"conv1")
    conv_layer2 = calc_conv_layer(conv_layer1,
        stride['conv2'],stride['pool2'],padding['conv2'],padding['pool2'],patch_size['conv2'],patch_size['pool2'],depth['conv2'],"conv2")
#     conv_layer3 = calc_conv_layer(conv_layer2,
#         stride['conv3'],stride['pool3'],padding['conv3'],padding['pool3'],patch_size['conv3'],patch_size['pool3'],depth['conv3'],"conv3")
    shape = conv_layer2.get_shape().as_list()
    # reshape cannot accept shape[0] whose value is None, that's why tf.shape(conv_layer3)[0] is used instead
    reshape = tf.reshape(conv_layer2, [tf.shape(conv_layer2)[0], shape[1] * shape[2] * shape[3]])
    hidden = calc_full_layer(reshape, num_hidden, keep_prob, True, "fc1")
    return calc_full_layer(hidden, num_labels, 1, False, "fc2")
    

  global_step = tf.Variable(0, trainable=False)

  # Training computation.
  logits = model(dataset, dropout_keep_prob)

  with tf.name_scope("loss"):
      loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits))
    
  learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                           100, 0.96, staircase=True)
  
  with tf.name_scope("optimizer"):
  # Optimizer.
      optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  prediction = tf.nn.softmax(logits)


In [59]:
writer = tf.summary.FileWriter("/root/logs/conv/2")
writer.add_graph(graph)

In [60]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    train_feed_dict = {dataset : batch_data, labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, prediction], feed_dict=train_feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      valid_feed_dict = {dataset : valid_dataset, labels : valid_labels}
      valid_prediction = session.run(prediction, feed_dict = valid_feed_dict)
      print('Validation accuracy: %.1f%%' % accuracy(valid_prediction, valid_labels))
  test_feed_dict = {dataset : test_dataset, labels : test_labels}
  test_prediction = session.run(prediction, feed_dict = test_feed_dict)
  print('Test accuracy: %.1f%%' % accuracy(test_prediction, test_labels))

Initialized
Minibatch loss at step 0: 2.578190
Minibatch accuracy: 12.5%
Validation accuracy: 11.7%
Minibatch loss at step 50: 1.089813
Minibatch accuracy: 75.0%
Validation accuracy: 58.0%
Minibatch loss at step 100: 1.279646
Minibatch accuracy: 62.5%
Validation accuracy: 71.1%
Minibatch loss at step 150: 0.923415
Minibatch accuracy: 68.8%
Validation accuracy: 77.7%
Minibatch loss at step 200: 0.657699
Minibatch accuracy: 75.0%
Validation accuracy: 76.5%
Minibatch loss at step 250: 0.912244
Minibatch accuracy: 81.2%
Validation accuracy: 79.1%
Minibatch loss at step 300: 0.473691
Minibatch accuracy: 75.0%
Validation accuracy: 79.4%
Minibatch loss at step 350: 0.119682
Minibatch accuracy: 93.8%
Validation accuracy: 80.0%
Minibatch loss at step 400: 0.684719
Minibatch accuracy: 75.0%
Validation accuracy: 79.2%
Minibatch loss at step 450: 0.322577
Minibatch accuracy: 93.8%
Validation accuracy: 81.3%
Minibatch loss at step 500: 0.631913
Minibatch accuracy: 87.5%
Validation accuracy: 81.6%
M